# **Import Data**

In [15]:
import pandas as pd
wkdirPath = 'C:\\Users\\kevin\\Downloads\\workingDirectory\\' # TODO: Input the working directory path here
datasetFolder = 'dataset\\'
modelFolder = 'model\\'
metricFolder = 'metric\\'

datasetPath = wkdirPath + datasetFolder
modelPath = wkdirPath + modelFolder
metricPath = wkdirPath + metricFolder

In [16]:
X_data = pd.read_csv(datasetPath + "X_fe.csv", header=[0, 1], index_col=0)
y_data = pd.read_csv(datasetPath + "y_4.csv", index_col=0)

In [17]:
y_data.head()

,label
0,0
1,1
2,1
3,0
4,0


In [18]:
X_data.head()

quantile                                                              \
        FP1       FPz       FP2        F7        F3        Fz        F4   
0  3.275757  3.171221  6.266855  3.505204  1.761071  3.587153  6.038462   
1  3.482556  4.307556  6.733288  3.605462  2.020469  3.719703  6.900743   
2  6.936589  8.235714  7.147687  6.368727  2.023022  5.579708  8.134004   
3  5.156643  4.917971  6.914794  6.584577  1.623989  5.219904  7.672572   
4  1.436962  1.301774  6.793815  3.620672  1.269546  2.655743  5.243719   

                                 ...    variance                          \
         F8       FT9       FC5  ...         CP2         CP6        TP10   
0  6.392021  4.244585  1.845967  ...   68.961700  129.270645  116.701103   
1  6.842275  4.416522  1.803723  ...   86.855766  151.384186  133.853088   
2  9.014104  5.933595  2.346217  ...  166.771744  399.105408  153.683090   
3  9.140758  5.750987  2.627729  ...  199.748566  297.561798  149.803162   
4  6.758463  4.352636  1.941540  ...   75.045914  171.823349  111.750275   

                                                                        \
          P7         P3         Pz          P4          P8          O1   
0  68.938408  61.535503  71.211472   90.592781  152.309875   97.097801   
1  77.523170  70.528954  84.750725  108.770752  167.561554  114.627052   
2  80.086365  64.642540  86.513268  123.233734  186.522812  107.269867   
3  84.879875  66.953796  90.916534  126.755249  181.390167  112.286552   
4  62.643578  57.428719  72.822304  106.469200  154.672363   95.603989   

               
           O2  
0  130.508011  
1  143.692932  
2  163.319839  
3  166.973053  
4  150.293564  

[5 rows x 192 columns]

In [19]:
# ! pip install xgboost
# ! pip install lightgbm

# **Model**

In [20]:
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
import sklearn 

from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

In [21]:
print('lightgbm version:', lgb.__version__)
print('xgboost version:', xgb.__version__)
print('catboost version:', cb.__version__)
print('sklearn version:', sklearn.__version__)

lightgbm version: 4.1.0
xgboost version: 2.0.3
catboost version: 1.2.2
sklearn version: 1.2.2


In [22]:
lightgbm = LGBMClassifier()
xgboost = XGBRFClassifier()
catboost = CatBoostClassifier()
histgb = HistGradientBoostingClassifier()

In [23]:
model_classifier_dict = {
    "histgb": histgb,
    "lgbm": lightgbm,
    "catb": catboost,
    "xgb": xgboost
}

In [24]:
from sklearn.metrics import (
    make_scorer,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
    accuracy_score,
    log_loss,
    mean_absolute_error,
    r2_score,
)
from sklearn.model_selection import cross_val_score, StratifiedKFold, cross_validate

In [25]:
# Evaluate our models
def evaluate_model(model_dict, X_train, y_train):
    scoring = {
        "accuracy": make_scorer(accuracy_score),
        "f1": make_scorer(f1_score, average="macro"),
        "precision": make_scorer(precision_score, average="macro"),
        "recall": make_scorer(recall_score, average="macro"),
        "roc_auc": make_scorer(roc_auc_score, needs_proba=True),
        "log_loss": make_scorer(log_loss, needs_proba=True),
        "mae": make_scorer(mean_absolute_error),
        "r2": make_scorer(r2_score),
    }

    kf = StratifiedKFold(n_splits=3)

    scores_dict = dict()
    for name, model in model_dict.items():
        # proba = cross_val_predict(model, X_train, y_train, cv=kf, method='predict_proba')
        scores = cross_validate(
            model, X_train, y_train, cv=kf, scoring=scoring, n_jobs=12, verbose=1
        )
        print("Model: ", name, "done.")
        scores_dict[name] = scores
    return scores_dict

In [26]:
scores_dict = evaluate_model(
    model_classifier_dict, X_data.values.copy(), y_data.values.copy()
)

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   3 out of   3 | elapsed:   13.8s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.


Model:  histgb done.


[Parallel(n_jobs=12)]: Done   3 out of   3 | elapsed:    6.0s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.


Model:  lgbm done.


[Parallel(n_jobs=12)]: Done   3 out of   3 | elapsed:  1.3min finished
c:\Python38\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
2 fits failed out of a total of 3.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Python38\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Python38\lib\site-packages\catboost\core.py", line 5100, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "c:\Python38\lib\site-packages\catboost\core.p

Model:  catb done.
Model:  xgb done.


[Parallel(n_jobs=12)]: Done   3 out of   3 | elapsed:    9.1s finished


In [27]:
scores_dict

{'histgb': {'fit_time': array([11.19313049, 11.28312421, 11.44813275]),
  'score_time': array([0.09700131, 0.04199553, 0.03398991]),
  'test_accuracy': array([0.32423208, 0.31399317, 0.36986301]),
  'test_f1': array([0.24016168, 0.16345679, 0.21460047]),
  'test_precision': array([0.29677318, 0.14638298, 0.18870656]),
  'test_recall': array([0.27230496, 0.18705882, 0.26475556]),
  'test_roc_auc': array([nan, nan, nan]),
  'test_log_loss': array([3.03297712, 3.32227322, 2.82979084]),
  'test_mae': array([1.06825939, 1.15017065, 0.96232877]),
  'test_r2': array([-0.61913866, -0.9368354 , -0.45077034])},
 'lgbm': {'fit_time': array([3.79699373, 3.78799081, 3.68499851]),
  'score_time': array([0.01599097, 0.01699162, 0.06298995]),
  'test_accuracy': array([0.32081911, 0.31399317, 0.38356164]),
  'test_f1': array([0.23313276, 0.17864747, 0.22242809]),
  'test_precision': array([0.26461664, 0.18157333, 0.19364224]),
  'test_recall': array([0.27229044, 0.19584436, 0.27518879]),
  'test_roc_au

In [28]:
def switch_nested_dict(nested_dict):
    switched_dict = {}

    for key1, dict2 in nested_dict.items():
        for key2, data in dict2.items():
            if key2 not in switched_dict:
                switched_dict[key2] = {}
            switched_dict[key2][key1] = data

    return switched_dict
scores_dict_switched = switch_nested_dict(scores_dict)

In [29]:
for rank in range(len(model_classifier_dict)):
    for title, scores in scores_dict_switched.items():
        tupl = sorted(scores.items(), key=lambda item: -item[1][2])[rank]
        print(f"{title} #{rank+1}:", tupl[0], "with score of", tupl[1][2])
    print()

fit_time #1: catb with score of 75.68765091896057
score_time #1: lgbm with score of 0.06298995018005371
test_accuracy #1: xgb with score of 0.4041095890410959
test_f1 #1: xgb with score of 0.22728264884650257
test_precision #1: xgb with score of 0.19550438596491226
test_recall #1: lgbm with score of 0.2751887917329094
test_roc_auc #1: histgb with score of nan
test_log_loss #1: histgb with score of 2.8297908350187053
test_mae #1: catb with score of 0.9794520547945206
test_r2 #1: histgb with score of -0.45077033728281046

fit_time #2: histgb with score of 11.448132753372192
score_time #2: catb with score of 0.042989253997802734
test_accuracy #2: lgbm with score of 0.3835616438356164
test_f1 #2: lgbm with score of 0.22242809102484573
test_precision #2: lgbm with score of 0.19364224137931035
test_recall #2: xgb with score of 0.27389705882352944
test_roc_auc #2: lgbm with score of nan
test_log_loss #2: lgbm with score of 2.240862751090143
test_mae #2: lgbm with score of 0.9691780821917808
t